### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dataclasses
import sys

import einops
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import torch
import tqdm.auto as tqdm
import transformer_lens.utils as tl_utils
from jaxtyping import Float
from tqdm.auto import tqdm
from transformer_lens import ActivationCache, HookedTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
from tuned_lens.nn import LogitLens, TunedLens
from tuned_lens.plotting import PredictionTrajectory

from pii import utils
from pii.prompt_data import LensData, PromptData

torch.set_grad_enabled(False)

### Load model / lenses

In [3]:
# You will need to login to huggingface first:
#   huggingface-cli login
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
hf_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    low_cpu_mem_usage=True,
)
logit_lens = LogitLens.from_model(hf_model).to("cuda")
tuned_lens = TunedLens.from_model_and_pretrained(hf_model).to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
tl_model = HookedTransformer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    hf_model=hf_model,
    device="cuda",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    tokenizer=tokenizer,
)
tl_model.generate("The capital of Germany is", max_new_tokens=20, temperature=0)

Using pad_token, but it is not set yet.


Loaded pretrained model meta-llama/Llama-2-7b-chat-hf into HookedTransformer


  0%|          | 0/20 [00:00<?, ?it/s]

'The capital of Germany is Berlin. Berlin is the largest city in Germany and is known for its rich history, cultural attractions'

### Develop prompts

In [5]:
def get_corr_prompt(question: str, correct: bool = True):
    return f"""\
[INST] <<SYS>>
You are an assistant who only responds with a single word. \
You only give {'correct' if correct else 'incorrect'} answers.
<</SYS>>
Complete the statment. {question} [/INST] \
"""


def get_dontsay_prompt(question: str, forbidden_word: str):
    return f"""\
[INST] <<SYS>>
You are an obedient assistant who only responds with a single word. \
You will never say the forbidden word "{forbidden_word}".
<</SYS>>

{question} [/INST] \
"""


question = "Lionel Messi plays the sport of"

In [6]:
for i, corr in enumerate([True, False]):
    prompt = get_corr_prompt(question, correct=corr)
    print(prompt)
    utils.get_top_responses(
        prompt=prompt,
        model=tl_model,
        top_k=5,
    )
    if i == 0:
        print()

[INST] <<SYS>>
You are an assistant who only responds with a single word. You only give correct answers.
<</SYS>>
Complete the statment. Lionel Messi plays the sport of [/INST] 
Rank 0. Logit: 25.57 Prob: 92.37% Tokens: ( 8914) |Football|</s>|
Rank 1. Logit: 23.00 Prob:  7.01% Tokens: (18993) |Soccer|</s>|
Rank 2. Logit: 19.99 Prob:  0.35% Tokens: ( 5733) |football|</s>|
Rank 3. Logit: 18.87 Prob:  0.11% Tokens: ( 7527) |Foot|bal|</s>|
Rank 4. Logit: 18.17 Prob:  0.06% Tokens: (13440) |Foot|ball|</s>|

[INST] <<SYS>>
You are an assistant who only responds with a single word. You only give incorrect answers.
<</SYS>>
Complete the statment. Lionel Messi plays the sport of [/INST] 
Rank 0. Logit: 14.06 Prob: 14.76% Tokens: (29010) |Tennis|</s>|
Rank 1. Logit: 13.86 Prob: 12.19% Tokens: (  402) |G|olf|.|</s>|
Rank 2. Logit: 13.64 Prob:  9.76% Tokens: (  349) |P|ing|-|p|ong|.|
Rank 3. Logit: 13.50 Prob:  8.49% Tokens: (21850) |Basketball|</s>|
Rank 4. Logit: 13.30 Prob:  6.91% Tokens: (  32

In [7]:
for i, forbidden_word in enumerate(["apple", "football"]):
    prompt = get_dontsay_prompt(question, forbidden_word)
    print(prompt)
    utils.get_top_responses(
        prompt=prompt,
        model=tl_model,
        top_k=5,
    )
    if i == 0:
        print()

[INST] <<SYS>>
You are an obedient assistant who only responds with a single word. You will never say the forbidden word "apple".
<</SYS>>

Lionel Messi plays the sport of [/INST] 
Rank 0. Logit: 23.57 Prob: 70.88% Tokens: ( 8914) |Football|</s>|
Rank 1. Logit: 22.62 Prob: 27.59% Tokens: (18993) |Soccer|</s>|
Rank 2. Logit: 18.48 Prob:  0.44% Tokens: ( 5733) |football|</s>|
Rank 3. Logit: 18.19 Prob:  0.33% Tokens: (20582) |Fut|bol|</s>|
Rank 4. Logit: 17.91 Prob:  0.25% Tokens: ( 7527) |Foot|bal|</s>|

[INST] <<SYS>>
You are an obedient assistant who only responds with a single word. You will never say the forbidden word "football".
<</SYS>>

Lionel Messi plays the sport of [/INST] 
Rank 0. Logit: 20.47 Prob: 98.34% Tokens: (18993) |Soccer|</s>|
Rank 1. Logit: 15.16 Prob:  0.49% Tokens: (  376) |"|S|occer|"|</s>|
Rank 2. Logit: 14.26 Prob:  0.20% Tokens: (21850) |Basketball|</s>|
Rank 3. Logit: 14.19 Prob:  0.18% Tokens: (23185) |Baseball|</s>|
Rank 4. Logit: 13.83 Prob:  0.13% Tokens

### Cache activations

In [8]:
prompt_dict = dict(
    corr_base=PromptData.get_data(get_corr_prompt(question), tl_model),
    corr_competing=PromptData.get_data(get_corr_prompt(question, correct=False), tl_model),
    dontsay_base=PromptData.get_data(get_dontsay_prompt(question, "apple"), tl_model),
    dontsay_competing=PromptData.get_data(get_dontsay_prompt(question, "football"), tl_model),
)

for prompt_type in ["corr", "dontsay"]:
    base_data = prompt_dict[f"{prompt_type}_base"]
    competing_data = prompt_dict[f"{prompt_type}_competing"]

    print(f"Logits for {prompt_type}_base")
    print(base_data.ml_str, base_data.ml_token, base_data.final_logits[base_data.ml_token].item())
    print(competing_data.ml_str, competing_data.ml_token, base_data.final_logits[competing_data.ml_token].item())
    print(f"Logits for {prompt_type}_competing")
    print(base_data.ml_str, base_data.ml_token, competing_data.final_logits[base_data.ml_token].item())
    print(competing_data.ml_str, competing_data.ml_token, competing_data.final_logits[competing_data.ml_token].item())

Logits for corr_base
Football 8914 25.573688507080078
Tennis 29010 10.12660026550293
Logits for corr_competing
Football 8914 12.586689949035645
Tennis 29010 14.05516242980957
Logits for dontsay_base
Football 8914 23.56771469116211
Soccer 18993 22.624164581298828
Logits for dontsay_competing
Football 8914 12.282051086425781
Soccer 18993 20.46957015991211


In [22]:
logits_base, cache_base = tl_model.run_with_cache(
    get_corr_prompt(question, correct=True),
    remove_batch_dim=True,
)
logits_competing, cache_competing = tl_model.run_with_cache(
    get_corr_prompt(question, correct=False),
    remove_batch_dim=True,
)

logits_base = tl_utils.remove_batch_dim(logits_base)
logits_competing = tl_utils.remove_batch_dim(logits_competing)
logits_base.shape, logits_competing.shape

(torch.Size([53, 32000]), torch.Size([53, 32000]))

### Compute lens data

In [9]:
lens_dict = {
    k: LensData.get_data(
        prompt_data=v,
        tl_model=tl_model,
        logit_lens=logit_lens,
        tuned_lens=tuned_lens,
    )
    for k, v in prompt_dict.items()
}

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


### Analyze attention

In [19]:
import numpy as np

def subject_relation_attention(
        cache: dict,
        subject_index: int,
        relation_index: int,
        correct_indices: (int, int),
):
    total_list = []
    for layer_num in range(32):
        head_list = []
        for head_num in range(32):
            relation_attention = np.sum(cache[f"blocks.{layer_num}.attn.hook_pattern"][head_num, -1, relation_index:].cpu().numpy())
            subject_attention = np.sum(cache[f"blocks.{layer_num}.attn.hook_pattern"][head_num, -1, subject_index:relation_index].cpu().numpy())
            correct_attention = np.sum(cache[f"blocks.{layer_num}.attn.hook_pattern"][head_num, -1, correct_indices[0]:correct_indices[1]].cpu().numpy())
            head_list.append((relation_attention, subject_attention, correct_attention))
        total_list.append(head_list)
    return total_list

def topk_heads_attention(total_list, k=1):
    # Flatten list and associate each attention value with its layer and head
    flattened_list = [((layer_num, head_num), head_list[2]) for layer_num, layer in enumerate(total_list) for head_num, head_list in enumerate(layer)]
    
    # Sort based on correct attention
    sorted_list = sorted(flattened_list, key=lambda x: x[1], reverse=True)
    
    # Return top k
    return sorted_list[:k]


In [24]:
# Example usage:
total_list = subject_relation_attention(cache_competing, 40, 44, (25, 26))
top_heads = topk_heads_attention(total_list, k=5)
print(top_heads)

[((9, 17), 0.3763387), ((15, 5), 0.3484298), ((13, 18), 0.29520547), ((11, 8), 0.22955495), ((11, 7), 0.21926266)]


### Split DLA by attention

In [17]:
from jaxtyping import Float
from einops import rearrange, reduce, repeat, einsum
def dla_attention_token_group(
    cache: ActivationCache,
    model: HookedTransformer,
    all_tokenized_attributes,
    bos_token_indices,
    subject_token_indices,
    relation_token_indices,
) -> Float[torch.Tensor, "attribute model_component token_group"]:
    """DLA by attention head, by token group that is attended to

    The token groups are (bos, subject, relation).
    """

    layers = []

    logit_directions: Float[
        torch.Tensor, "attribute d_model"
    ] = model.tokens_to_residual_directions(torch.tensor(all_tokenized_attributes))

    for layer in range(model.cfg.n_layers):
        value: Float[torch.Tensor, "pos head_index d_head"] = cache[
            f"blocks.{layer}.attn.hook_v"
        ][0]

        pattern_post_softmax: Float[
            torch.Tensor, "head_index query_pos key_pos"
        ] = cache[f"blocks.{layer}.attn.hook_pattern"][0]

        # Z is usually calculated as values * attention, summed across keys
        z = einsum(
            "key_pos head_index d_head, \
                head_index query_pos key_pos -> \
                query_pos key_pos head_index d_head",
            value,
            pattern_post_softmax,
        )

        weights_output: Float[torch.Tensor, "head_index d_head d_model"] = model.blocks[
            layer
        ].attn.W_O

        # bias_output: Float[torch.Tensor, "d_model"] = model.blocks[layer].attn.b_O

        result = einsum(
            "query_pos key_pos head_index d_head, \
                    head_index d_head d_model -> \
                    query_pos key_pos head_index d_model",
            z,
            weights_output,
        )

        # Stacked head result (pos slice on last token)
        result_on_final_token: Float[
            torch.Tensor, "key_pos head_index d_model"
        ] = result[-1]

        # Scale
        if model.cfg.normalization_type not in ["LN", "LNPre"]:
            scaled_result = result_on_final_token
        else:
            center_stack = result_on_final_token - result_on_final_token.mean(
                dim=-1, keepdim=True
            )
            scale = cache["ln_final.hook_scale"][0, -1, :]  # first batch, last token
            scaled_result = center_stack / scale

        logit_attrs = einsum(
            "key_pos head_index d_model, attribute d_model -> key_pos head_index attribute",
            scaled_result,
            logit_directions,
        )

        logit_attrs = einops.rearrange(
            logit_attrs, "key_pos head_index attribute -> attribute head_index key_pos"
        )

        # Sum by category (bos, subject, relation)
        bos_token_attribution: Float[
            torch.Tensor, "attribute head_index"
        ] = logit_attrs[:, :, bos_token_indices].sum(dim=-1)
        subject_tokens_attribution: Float[
            torch.Tensor, "attribute head_index"
        ] = logit_attrs[:, :, subject_token_indices].sum(dim=-1)
        relation_token_attribution: Float[
            torch.Tensor, "attribute head_index"
        ] = logit_attrs[:, :, relation_token_indices].sum(dim=-1)

        combined: Float[torch.Tensor, "attribute head_index token_group"] = torch.stack(
            [
                bos_token_attribution,
                subject_tokens_attribution,
                relation_token_attribution,
            ],
            dim=-1,
        )
        layers.append(combined)

    layers_tensor: Float[
        torch.Tensor, "layer attribute head_index token_group"
    ] = torch.stack(layers, dim=0)

    rearranged = einops.rearrange(
        layers_tensor,
        "layer attribute head_index token_group -> attribute (layer head_index) token_group",
    )

    return rearranged

### Plot by head

In [14]:
ld_base = lens_dict["corr_base"]
ld_comp = lens_dict["corr_competing"]

logits_base = ld_base.get_logits("attn_head", lens_type="tuned")
logits_comp = ld_comp.get_logits("attn_head", lens_type="tuned")

utils.plot_head_data(
    lines=[
        (
            f"logit: |{ld_base.pd.ml_str}|", 
            logits_base[:, ld_base.pd.ml_token]
        ),
        (
            f"logit: |{ld_comp.pd.ml_str}|", 
            logits_base[:, ld_comp.pd.ml_token]
        ),
    ],
    annotation_text=ld_base.pd.prompt.split("\n")[-1],
    title="Answer correctly prompt",
    yaxis_title="Logit",
).show()

utils.plot_head_data(
    lines=[
        (
            f"logit: |{ld_base.pd.ml_str}|", 
            logits_comp[:, ld_base.pd.ml_token]
        ),
        (
            f"logit: |{ld_comp.pd.ml_str}|", 
            logits_comp[:, ld_comp.pd.ml_token]
        ),
    ],
    annotation_text=ld_comp.pd.prompt.split("\n")[-1],
    title="Answer incorrectly prompt",
    yaxis_title="Logit",
).show()

In [16]:
ld_base = lens_dict["dontsay_base"]
ld_comp = lens_dict["dontsay_competing"]

logits_base = ld_base.get_logits("attn_head", lens_type="tuned")
logits_comp = ld_comp.get_logits("attn_head", lens_type="tuned")

utils.plot_head_data(
    lines=[
        (
            f"logit: |{ld_base.pd.ml_str}|", 
            logits_base[:, ld_base.pd.ml_token]
        ),
        (
            f"logit: |{ld_comp.pd.ml_str}|", 
            logits_base[:, ld_comp.pd.ml_token]
        ),
    ],
    annotation_text=ld_base.pd.prompt.split("\n")[-1],
    title="Don't say apple",
    yaxis_title="Logit",
).show()

utils.plot_head_data(
    lines=[
        (
            f"logit: |{ld_base.pd.ml_str}|", 
            logits_comp[:, ld_base.pd.ml_token]
        ),
        (
            f"logit: |{ld_comp.pd.ml_str}|", 
            logits_comp[:, ld_comp.pd.ml_token]
        ),
    ],
    annotation_text=ld_comp.pd.prompt.split("\n")[-1],
    title="Don't say football",
    yaxis_title="Logit",
).show()

## Attention plots

In [ ]:
str_tokens = tl_model.to_str_tokens(get_corr_prompt(question, correct=False))

In [ ]:
str_tokens = tl_model.to_str_tokens(get_corr_prompt(question, correct=False))

In [ ]:
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Neel")

In [ ]:
cache_incorrect.keys()

In [ ]:
for i in range(31, -1, -1):
    attention_pattern = cache_incorrect[f"blocks.{i}.attn.hook_pattern"]

    print(attention_pattern.shape)
    print(f"Layer {i} Head Attention Patterns:")
    display(cv.attention.attention_patterns(tokens=str_tokens, attention=attention_pattern))